In [1]:
import sparse
import numba
import numpy as np
import dask.distributed as dd
import libertem.api as lt
import libertem.masks as masks

In [2]:
client = dd.Client(n_workers=2, threads_per_worker=1)

In [3]:
def do_numpy():
    a = np.arange(1024*1024, dtype=np.float32)
    return np.dot(a, a)

In [4]:
@numba.njit
def do_numba():
    result = 0.
    for i in range(1024*1024):
        result += i**2
    return result

In [5]:
def do_sparse():
    a = sparse.COO.from_numpy(np.arange(1024*1024, dtype=np.float32))
    return sparse.dot(a, a)

In [6]:
%time do_numpy()

Wall time: 8 ms


3.843066e+17

In [7]:
%time do_numpy()

Wall time: 12 ms


3.843066e+17

In [8]:
%time do_numba()

Wall time: 672 ms


3.843066184463922e+17

In [9]:
%time do_numba()

Wall time: 4 ms


3.843066184463922e+17

In [10]:
%time do_sparse()

Wall time: 7.98 s


3.843066e+17

In [11]:
%time do_sparse()

Wall time: 1.03 s


3.843066e+17

In [12]:
f = client.submit(do_numpy)
%time f.result()

Wall time: 296 ms


3.843066e+17

In [13]:
f = client.submit(do_numpy)
%time f.result()

Wall time: 28 ms


3.843066e+17

In [14]:
f = client.submit(do_numba)
%time f.result()

Wall time: 1.2 s


3.843066184463922e+17

In [15]:
f = client.submit(do_numba)
%time f.result()

Wall time: 36 ms


3.843066184463922e+17

In [16]:
f = client.submit(do_sparse)
%time f.result()

Wall time: 9.33 s


3.843066e+17

In [17]:
f = client.submit(do_sparse)
%time f.result()

Wall time: 24 ms


3.843066e+17

In [18]:
ctx = lt.Context()
ds = ctx.load(
    "blo",
    path='C:/Users/weber/Nextcloud/Projects/Open Pixelated STEM framework/Data/3rd-party Datasets/Glasgow/10 um 110.blo',
    tileshape=(1,8,144,144)
)

In [19]:
analysis_sparse_1 = ctx.create_disk_analysis(dataset=ds, r=1)

In [20]:
%time ctx.run(analysis_sparse_1)

Wall time: 5.78 s


[<AnalysisResult: intensity>]

In [21]:
%time ctx.run(analysis_sparse_1)

Wall time: 1.66 s


[<AnalysisResult: intensity>]

In [22]:
analysis_sparse_2 = ctx.create_disk_analysis(dataset=ds, r=2)

In [23]:
%time ctx.run(analysis_sparse_2)

Wall time: 1.65 s


[<AnalysisResult: intensity>]

In [24]:
%time ctx.run(analysis_sparse_2)

Wall time: 1.67 s


[<AnalysisResult: intensity>]

In [25]:
analysis_dense = ctx.create_disk_analysis(dataset=ds, r=256)

In [26]:
%time ctx.run(analysis_dense)

Wall time: 2.53 s


[<AnalysisResult: intensity>]

In [27]:
%time ctx.run(analysis_dense)

Wall time: 1.97 s


[<AnalysisResult: intensity>]

In [28]:
fy, fx = ds.shape.sig

In [29]:
mask_stack = [
    lambda: masks.circular(centerY=(i%fy), centerX=i%fx, imageSizeY=fy, imageSizeX=fx, radius=2)
    for i in range(1000)
]

In [30]:
multi_analysis_sparse = ctx.create_mask_analysis(dataset=ds, factories=mask_stack, use_sparse=True)
multi_analysis_dense = ctx.create_mask_analysis(dataset=ds, factories=mask_stack, use_sparse=False)

In [31]:
%time ctx.run(multi_analysis_sparse)

Wall time: 19.4 s


[<AnalysisResult: mask_0>, <AnalysisResult: mask_1>, <AnalysisResult: mask_2>, <AnalysisResult: mask_3>, <AnalysisResult: mask_4>, <AnalysisResult: mask_5>, <AnalysisResult: mask_6>, <AnalysisResult: mask_7>, <AnalysisResult: mask_8>, <AnalysisResult: mask_9>, <AnalysisResult: mask_10>, <AnalysisResult: mask_11>, <AnalysisResult: mask_12>, <AnalysisResult: mask_13>, <AnalysisResult: mask_14>, <AnalysisResult: mask_15>, <AnalysisResult: mask_16>, <AnalysisResult: mask_17>, <AnalysisResult: mask_18>, <AnalysisResult: mask_19>, <AnalysisResult: mask_20>, <AnalysisResult: mask_21>, <AnalysisResult: mask_22>, <AnalysisResult: mask_23>, <AnalysisResult: mask_24>, <AnalysisResult: mask_25>, <AnalysisResult: mask_26>, <AnalysisResult: mask_27>, <AnalysisResult: mask_28>, <AnalysisResult: mask_29>, <AnalysisResult: mask_30>, <AnalysisResult: mask_31>, <AnalysisResult: mask_32>, <AnalysisResult: mask_33>, <AnalysisResult: mask_34>, <AnalysisResult: mask_35>, <AnalysisResult: mask_36>, <AnalysisR

In [32]:
%time ctx.run(multi_analysis_dense)

Wall time: 49.3 s


[<AnalysisResult: mask_0>, <AnalysisResult: mask_1>, <AnalysisResult: mask_2>, <AnalysisResult: mask_3>, <AnalysisResult: mask_4>, <AnalysisResult: mask_5>, <AnalysisResult: mask_6>, <AnalysisResult: mask_7>, <AnalysisResult: mask_8>, <AnalysisResult: mask_9>, <AnalysisResult: mask_10>, <AnalysisResult: mask_11>, <AnalysisResult: mask_12>, <AnalysisResult: mask_13>, <AnalysisResult: mask_14>, <AnalysisResult: mask_15>, <AnalysisResult: mask_16>, <AnalysisResult: mask_17>, <AnalysisResult: mask_18>, <AnalysisResult: mask_19>, <AnalysisResult: mask_20>, <AnalysisResult: mask_21>, <AnalysisResult: mask_22>, <AnalysisResult: mask_23>, <AnalysisResult: mask_24>, <AnalysisResult: mask_25>, <AnalysisResult: mask_26>, <AnalysisResult: mask_27>, <AnalysisResult: mask_28>, <AnalysisResult: mask_29>, <AnalysisResult: mask_30>, <AnalysisResult: mask_31>, <AnalysisResult: mask_32>, <AnalysisResult: mask_33>, <AnalysisResult: mask_34>, <AnalysisResult: mask_35>, <AnalysisResult: mask_36>, <AnalysisR

In [33]:
indices = np.arange(1000, dtype=int)
centersY = indices % fy
centersX = indices % fx

In [34]:
stack_factory = lambda: masks.sparse_circular_multi_stack(indices, centersX, centersY, fx, fy, 2)

In [35]:
stack_analysis_sparse = ctx.create_mask_analysis(dataset=ds, factories=stack_factory, length=len(indices))
stack_analysis_dense = ctx.create_mask_analysis(dataset=ds, factories=stack_factory, length=len(indices), use_sparse=False)

In [36]:
%time ctx.run(stack_analysis_sparse)

Wall time: 13.3 s


[<AnalysisResult: mask_0>, <AnalysisResult: mask_1>, <AnalysisResult: mask_2>, <AnalysisResult: mask_3>, <AnalysisResult: mask_4>, <AnalysisResult: mask_5>, <AnalysisResult: mask_6>, <AnalysisResult: mask_7>, <AnalysisResult: mask_8>, <AnalysisResult: mask_9>, <AnalysisResult: mask_10>, <AnalysisResult: mask_11>, <AnalysisResult: mask_12>, <AnalysisResult: mask_13>, <AnalysisResult: mask_14>, <AnalysisResult: mask_15>, <AnalysisResult: mask_16>, <AnalysisResult: mask_17>, <AnalysisResult: mask_18>, <AnalysisResult: mask_19>, <AnalysisResult: mask_20>, <AnalysisResult: mask_21>, <AnalysisResult: mask_22>, <AnalysisResult: mask_23>, <AnalysisResult: mask_24>, <AnalysisResult: mask_25>, <AnalysisResult: mask_26>, <AnalysisResult: mask_27>, <AnalysisResult: mask_28>, <AnalysisResult: mask_29>, <AnalysisResult: mask_30>, <AnalysisResult: mask_31>, <AnalysisResult: mask_32>, <AnalysisResult: mask_33>, <AnalysisResult: mask_34>, <AnalysisResult: mask_35>, <AnalysisResult: mask_36>, <AnalysisR

In [37]:
%time ctx.run(stack_analysis_dense)

Wall time: 50.7 s


[<AnalysisResult: mask_0>, <AnalysisResult: mask_1>, <AnalysisResult: mask_2>, <AnalysisResult: mask_3>, <AnalysisResult: mask_4>, <AnalysisResult: mask_5>, <AnalysisResult: mask_6>, <AnalysisResult: mask_7>, <AnalysisResult: mask_8>, <AnalysisResult: mask_9>, <AnalysisResult: mask_10>, <AnalysisResult: mask_11>, <AnalysisResult: mask_12>, <AnalysisResult: mask_13>, <AnalysisResult: mask_14>, <AnalysisResult: mask_15>, <AnalysisResult: mask_16>, <AnalysisResult: mask_17>, <AnalysisResult: mask_18>, <AnalysisResult: mask_19>, <AnalysisResult: mask_20>, <AnalysisResult: mask_21>, <AnalysisResult: mask_22>, <AnalysisResult: mask_23>, <AnalysisResult: mask_24>, <AnalysisResult: mask_25>, <AnalysisResult: mask_26>, <AnalysisResult: mask_27>, <AnalysisResult: mask_28>, <AnalysisResult: mask_29>, <AnalysisResult: mask_30>, <AnalysisResult: mask_31>, <AnalysisResult: mask_32>, <AnalysisResult: mask_33>, <AnalysisResult: mask_34>, <AnalysisResult: mask_35>, <AnalysisResult: mask_36>, <AnalysisR